In [1]:
# MaC 혹은 Linux 는 요렇게
import time
import multiprocessing

def count_down(x):
    while True:
        if x == 0:
            break
            
        print("카운트 다운 ... %d" % x)
        x -= 1
        time.sleep(1)
        
p = multiprocessing.Process(target=count_down, args=(5,))
p.start()

In [6]:
# Windows 는 요렇게
import time
import multiprocessing

def count_down(x):
    while True:
        if x == 0:
            break
            
        print("카운트 다운 ... %d" % x)
        x -= 1
        time.sleep(1)

if __name__ == '__main__':
    p = multiprocessing.Process(target=count_down, args=(5,))
    print(p)
    p.start()
    p.join()
    
#parent숫자는 PID 번호임

<Process name='Process-2' parent=13048 initial>


In [ ]:
#프로세스 리스트
#아래거 안되면 !ps -alx
# !ps -ef

In [ ]:
#Task 가 CPU의 추상화인 이유는?
#동시에 작동한다? ---> 정말 동시에 작동하는 것이 아니다!
#Multi-Tasking(멀티 태스킹) ---> 실제로는 시분할로 동작
#실제로는 우리가 구동시키는 모든 프로그램들은 전부 다 하나도 빠짐없이 모두 기계어로 구동된다.
#이 기계어를 구동시킬 수 있는 녀석은 유일하게 CPU뿐이다.
#결국 이 프로그램들이 시분할로 동작하면서 경쟁하는 이유는? ---> CPU자원이 한정되어 있기 때문이다.

In [13]:
import threading
import os

def info():
    print('module name: ', __name__)
    print('parant process: ', os.getppid())
    print('process id:', os.getpid())
    print('thread id: ', threading.get_ident())
    print('이런식으로 많이 집어넣으면 Context Switching 도 직접 확인 가능')

def print_hello():
    print("안녕!")
    info()
    time.sleep(1) #이게 들어가니까 context switching 무조건 들어가게 됨.
    print("잠자다 일어나기\n")
    
if __name__ == '__main__':
    start = time.perf_counter()
    threads = []
    
    for _ in range(5):
        t = threading.Thread(target = print_hello)
        t.start()
        threads.append(t)
        
    for thread in threads:
        thread.join()
        
    finish = time.perf_counter()
    
    print('종료: ', str(round(finish - start, 2)) + "초")
    
    
#결과는 매번 달라질 수 밖에 없다. WHY? --> multi tasking 할 때 시분할이 매번 달라지기 때문!
#async와 await가 없으니 동작 끝나는 걸 기다려주지 않음.

안녕!
module name:  __main__
parant process:  18352
process id: 13048
thread id:  19628
이런식으로 많이 집어넣으면 Context Switching 도 직접 확인 가능
안녕!
module name:  __main__
parant process:  18352
process id: 13048
thread id:  18684
이런식으로 많이 집어넣으면 Context Switching 도 직접 확인 가능
안녕!
module name:  __main__
parant process:  18352
process id: 13048
thread id:  19504
이런식으로 많이 집어넣으면 Context Switching 도 직접 확인 가능
안녕!
module name:  __main__
parant process:  18352
process id: 13048
thread id:  19476
이런식으로 많이 집어넣으면 Context Switching 도 직접 확인 가능
안녕!
module name:  __main__
parant process:  18352
process id: 13048
thread id:  19480
이런식으로 많이 집어넣으면 Context Switching 도 직접 확인 가능
잠자다 일어나기

잠자다 일어나기

잠자다 일어나기
잠자다 일어나기


잠자다 일어나기

종료:  1.06초


In [14]:
#크리티컬 섹션에서 발생하는 문제를 확인하기 위해 만든 예제.

import threading

x = 0 #크리티컬 섹션

def increment_global():
    global x
    x += 1
    
def thread_processing():
    for _ in range(1000000):
        increment_global()
        
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = thread_processing)
    t2 = threading.Thread(target = thread_processing)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
for i in range(10):
    thread_main()
    print("{0} 번째 반복 이후 x = {1}".format(i, x))

0 번째 반복 이후 x = 1728255
1 번째 반복 이후 x = 1644751
2 번째 반복 이후 x = 1572860
3 번째 반복 이후 x = 1529695
4 번째 반복 이후 x = 1402119
5 번째 반복 이후 x = 1824021
6 번째 반복 이후 x = 1846849
7 번째 반복 이후 x = 1672455
8 번째 반복 이후 x = 1581650
9 번째 반복 이후 x = 1652476


In [16]:
#크리티컬 섹션에서 발생하는 문제를 확인하기 위해 만든 예제.
  #위 문제에서 실행을 보장해주는 방식으로 코드를 변경하여 문제를 해결.
    #하지만 lock을 풀었다 잠그는 걸 반복하느라 실행속도가 느려졌다. 당연히 context switcing도 여러번 발생한다.

import threading

lock = threading.Lock()

x = 0 #크리티컬 섹션

def increment_global():
    global x
    x += 1
    
def thread_processing():
    for _ in range(1000000):
        lock.acquire()
        increment_global()
        lock.release()
        
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = thread_processing)
    t2 = threading.Thread(target = thread_processing)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
for i in range(10):
    thread_main()
    print("{0} 번째 반복 이후 x = {1}".format(i, x))

0 번째 반복 이후 x = 2000000
1 번째 반복 이후 x = 2000000
2 번째 반복 이후 x = 2000000
3 번째 반복 이후 x = 2000000
4 번째 반복 이후 x = 2000000
5 번째 반복 이후 x = 2000000
6 번째 반복 이후 x = 2000000
7 번째 반복 이후 x = 2000000
8 번째 반복 이후 x = 2000000
9 번째 반복 이후 x = 2000000


In [23]:
#크리티컬 섹션에서 발생하는 문제를 확인하기 위해 만든 예제.
  #위 문제에서 실행을 보장해주는 방식으로 코드를 변경하여 문제를 해결.
    #하지만 lock을 풀었다 잠그는 걸 반복하느라 실행속도가 느려졌다.
    #위의 실행속도 문제에서 발생한 비효율적인 문제를 해결하고 성능을 올린 예제이다.
    #같은 코드지만 다른 결과를 낼 수 있도록 하자. 리팩토링이 중요한 이유!

import threading

lock = threading.Lock()

x = 0 #크리티컬 섹션

def increment_global():
    global x
    x += 1
    
def thread_processing():
    lock.acquire()

    for _ in range(1000000):
        increment_global()
    
    lock.release()
        
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = thread_processing)
    t2 = threading.Thread(target = thread_processing)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
for i in range(10):
    thread_main()
    print("{0} 번째 반복 이후 x = {1}".format(i, x))
    
    #앞으로 Python에서 크롤링 등을 할 때는 Thread를 사용하게 된다. 잘 복습해둘 것

0 번째 반복 이후 x = 2000000
1 번째 반복 이후 x = 2000000
2 번째 반복 이후 x = 2000000
3 번째 반복 이후 x = 2000000
4 번째 반복 이후 x = 2000000
5 번째 반복 이후 x = 2000000
6 번째 반복 이후 x = 2000000
7 번째 반복 이후 x = 2000000
8 번째 반복 이후 x = 2000000
9 번째 반복 이후 x = 2000000


In [ ]:
#그러나 위의 코드에도 문제가 없지는 않음.
  #thread_processing()의 작업이 끝날 때 까지는 다른 쓰레드가 동작할 수가 없다.
    #cpu 자원을 잘 배분하는 것이 중요함.
  #때문에 잘못쓰면 속도가 느려질 수 있으므로 어떤 식으로 lock 장치를 사용할 것인지 고민해야 한다.

In [24]:
#Python을 이용한 DB처리
#MySQL with Python
!pip install pymysql

In [30]:
import pymysql

In [31]:
MYSQL_USER_DATA_SAVED_FILE = "mysql/userinfo"

In [33]:
import pickle

mysql_user_info = dict({
    'user_id': 'eddi',
    'password': 'eddi@123'
})

f = open(MYSQL_USER_DATA_SAVED_FILE, 'wb')
pickle.dump(mysql_user_info, f)
f.close()

del mysql_user_info

In [34]:
import pickle

f = open(MYSQL_USER_DATA_SAVED_FILE, 'rb')
mysql_user_info = pickle.load(f)
f.close()

In [35]:
db = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = mysql_user_info['user_id'],
    passwd = mysql_user_info['password'],
    # 여기서 사용할 스키마 하나 생성!
    db = 'pydb'
)

print(db)

In [36]:
cursor = db.cursor()

sql = """
    create table pydb_test(
    id int unsigned not null auto_increment,
    name varchar(20) not null,
    price int not null,
    primary key(id)
    )
"""

cursor.execute(sql) #실행
db.commit()
db.close()
print('테이블 생성 성공!')

테이블 생성 성공!
